In [13]:
### Import Dependencies


from gtts import gTTS
from moviepy.editor import *
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os
import csv
import tempfile
import shutil
from contextlib import contextmanager

import random
import time
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle


from pathlib import Path


# The code below takes a flashcard dataset in the form of a CSV file with a question column and an answer column, 
# uses the gTTS python library to convert the text to speech,
# and creates a video flashcard file showing both text and audio for the question and answer pairs.

In [7]:


# def preprocess_text_for_tts(self, text, debug=False):
#     """Convert mathematical notation to speakable text"""
    
#     # ... all the replacement code ...
    
#     if debug:
#         print(f"Original: {text}")
#         print(f"Spoken:   {result}")
#         print("-" * 40)
    
#     return result.strip()

class FlashcardVideoCreator:
    def __init__(self, width=1920, height=1080):
        self.width = width
        self.height = height
        self.font_size_question = 80
        self.font_size_answer = 100
        self.temp_dir = None
        
        # Setup fonts on initialization
        self.setup_fonts()

    def preprocess_text_for_tts(self, text):
        """Convert mathematical notation to speakable text for TTS"""
        
        # Handle special patterns with regex FIRST (before replacements)
        import re
        
        # Handle square root symbol to ensure proper spacing
        # Replace √ followed by anything with "square root of " plus that content
        result = re.sub(r'√(.)', r'square root of \1', text)
        
        # Handle minus vs negative
        # Replace negative numbers (dash immediately followed by digit)
        # Matches: -5, (-3, =-2, ^-1, or at start of string
        result = re.sub(r'(^|[\s\(=\^])-(\d)', r'\1negative \2', result)
        
        # Replace minus sign with spaces around it
        # This should come after negative replacement
        result = re.sub(r'\s-\s', ' minus ', result)
        
        # Handle cases like "x-1" or "n-1" (common in statistics)
        # Single letter followed by dash and number is usually minus
        result = re.sub(r'([a-zA-Z])-(\d)', r'\1 minus \2', result)
        
        # Dictionary of replacements - order matters for some replacements!
        replacements = {
            # Greek letters (uppercase)
            'Α': 'Alpha', 'Β': 'Beta', 'Γ': 'Gamma', 'Δ': 'Delta',
            'Σ': 'Sigma', 'Π': 'Pi', 'Ω': 'Omega',
            
            # Greek letters (lowercase)
            'α': 'alpha', 'β': 'beta', 'γ': 'gamma', 'δ': 'delta',
            'ε': 'epsilon', 'θ': 'theta', 'λ': 'lambda', 'μ': 'mu',
            'ν': 'nu', 'π': 'pi', 'ρ': 'rho', 'σ': 'sigma',
            'τ': 'tau', 'φ': 'phi', 'χ': 'chi', 'ψ': 'psi', 'ω': 'omega',
            
            # Common statistical notations (do these before subscripts)
            'H₀': 'H zero', 'H₁': 'H one', 'Hₐ': 'H A',
            'H0': 'H zero', 'H1': 'H one', 'Ha': 'H A',
            
            # Subscripts
            '₀': ' zero', '₁': ' one', '₂': ' two', '₃': ' three',
            '₄': ' four', '₅': ' five', '₆': ' six', '₇': ' seven',
            '₈': ' eight', '₉': ' nine',
            
            # Superscripts
            '²': ' squared', '³': ' cubed', '⁴': ' to the fourth',
            '⁻¹': ' to the negative 1', '⁻²': ' to the negative 2',
            
            # Mathematical operators
            '≤': 'less than or equal to',
            '≥': 'greater than or equal to',
            '≠': 'not equal to',
            '<': 'less than',
            '>': 'greater than',
            '±': 'plus or minus',
            '×': 'times',
            '÷': 'divided by',
            '≈': 'approximately',
            '∝': 'proportional to',
            '∈': 'element of',
            '∞': 'infinity',
            '∑': 'sum of',
            '∏': 'product of',
            '∫': 'integral of',
            
            # Common expressions
            # '√': 'square root of',  # REMOVED - now handled by regex above
            'σ²': 'sigma squared',
            'μ₁': 'mu 1', 'μ₂': 'mu 2', 'μ₃': 'mu 3',
            
            # Fractions and mathematical expressions
            '/': ' divided by ',
            '^': ' to the power of ',
            
            # Statistical terms
            'SD': 'standard deviation',
            'SE': 'standard error',
            'CI': 'confidence interval',
            'DF': 'degrees of freedom',
            'df': 'degrees of freedom',
            'ANOVA': 'ANOVA or analysis of variance',
            'i.e.': 'that is',
            'e.g.': 'for example',
            
            # Common statistical values
            'p-value': 'p value',
            'z-score': 'z score',
            't-statistic': 't statistic',
            'F-statistic': 'F statistic',
            'R²': 'R squared',
            'r²': 'r squared',
            'x̄': 'x bar',
            
            # Parenthetical clarifications for TTS
            '(': ', ',
            ')': ', ',
            '[': ', ',
            ']': ', ',
        }
        
        # Apply replacements to the result from regex processing
        for old, new in replacements.items():
            result = result.replace(old, new)
        
        # Additional regex patterns after replacements
        # Convert p = 0.05 style to "p equals 0.05"
        result = re.sub(r'\s*=\s*', ' equals ', result)
        
        # Convert "n=30" to "n equals 30"
        result = re.sub(r'([a-zA-Z])=(\d+)', r'\1 equals \2', result)
        
        # Convert percentages: "95%" to "95 percent"
        result = re.sub(r'(\d+)%', r'\1 percent', result)
        
        # Convert decimals for clarity: "0.05" to "zero point zero five"
        def decimal_to_words(match):
            decimal = match.group(0)
            if decimal == "0.01":
                return "zero point zero one"
            elif decimal == "0.05":
                return "zero point zero five"
            elif decimal == "0.10":
                return "zero point one zero"
            elif decimal == "0.95":
                return "zero point nine five"
            else:
                return decimal  # Leave others as is
        
        result = re.sub(r'0\.\d{2}', decimal_to_words, result)
        
        # Clean up multiple spaces
        result = re.sub(r'\s+', ' ', result)
        
        return result.strip()



    
    @contextmanager
    def temp_directory(self):
        """Context manager for temporary directory that cleans up automatically"""
        self.temp_dir = tempfile.mkdtemp(prefix="flashcard_video_")
        try:
            yield self.temp_dir
        finally:
            # Always clean up, even if there's an error
            if os.path.exists(self.temp_dir):
                shutil.rmtree(self.temp_dir)
                
    def setup_fonts(self):
        """Setup fonts - simplified to just use Windows fonts"""
        self.use_windows_fonts()
    

    def use_windows_fonts(self):
        """Use Windows system fonts with better Unicode support"""
        windows_fonts = [
            # Fonts with better Unicode coverage
            ("C:/Windows/Fonts/cambriab.ttf", "C:/Windows/Fonts/cambria.ttf"),  # Cambria
            ("C:/Windows/Fonts/DejaVuSans-Bold.ttf", "C:/Windows/Fonts/DejaVuSans.ttf"),  # DejaVu if installed
            ("C:/Windows/Fonts/segoeuib.ttf", "C:/Windows/Fonts/segoeui.ttf"),  # Segoe UI
            ("C:/Windows/Fonts/arialuni.ttf", "C:/Windows/Fonts/arialuni.ttf"),  # Arial Unicode if available
            # Fallback to Arial
            ("C:/Windows/Fonts/arialbd.ttf", "C:/Windows/Fonts/arial.ttf"),
        ]

        
        font_loaded = False
        for bold_path, regular_path in windows_fonts:
            if os.path.exists(bold_path) or os.path.exists(regular_path):
                font_to_use = bold_path if os.path.exists(bold_path) else regular_path
                regular_to_use = regular_path if os.path.exists(regular_path) else font_to_use
                
                try:
                    self.font_question_large = ImageFont.truetype(font_to_use, self.font_size_question)
                    self.font_question_small = ImageFont.truetype(regular_to_use, 40)
                    self.font_answer_large = ImageFont.truetype(font_to_use, self.font_size_answer)
                    self.font_answer_small = ImageFont.truetype(regular_to_use, 50)
                    self.font_countdown = ImageFont.truetype(font_to_use, 200)
                    self.font_countdown_text = ImageFont.truetype(regular_to_use, 40)
                    self.font_intro = ImageFont.truetype(font_to_use, 100)
                    print(f"✓ Using Windows font: {os.path.basename(font_to_use)}")
                    font_loaded = True
                    break
                except:
                    continue
        
        if not font_loaded:
            print("⚠ Warning: Using default font (will be small)")
            self.font_question_large = ImageFont.load_default()
            self.font_question_small = ImageFont.load_default()
            self.font_answer_large = ImageFont.load_default()
            self.font_answer_small = ImageFont.load_default()
            self.font_countdown = ImageFont.load_default()
            self.font_countdown_text = ImageFont.load_default()
            self.font_intro = ImageFont.load_default()
    
    def create_study_video(self, flashcards, output_file="study_video.mp4"):
        """Create a complete study video with visual flashcards"""
        
        with self.temp_directory():
            clips = []
            timestamps = []
            current_time = 0
            
            # Add intro clip
            print("Creating intro...")
            intro_clip = self.create_text_clip("Study Session Starting", duration=3)
            clips.append(intro_clip)
            current_time += 3
            
            total_cards = len(flashcards)
            for i, card in enumerate(flashcards):
                print(f"Processing card {i+1}/{total_cards}...")
                
                # Create question clip with dynamic duration
                q_clip = self.create_question_clip(card['question'], i+1)
                clips.append(q_clip)
                timestamps.append(f"{self.format_time(current_time)} Question {i+1}: {card['question'][:50]}")
                current_time += q_clip.duration
                
                # Create thinking pause
                pause_clip = self.create_pause_clip(duration=3)  # Reduced from 4
                clips.append(pause_clip)
                current_time += pause_clip.duration
                
                # Create answer clip with dynamic duration
                a_clip = self.create_answer_clip(card['answer'], i+1)
                clips.append(a_clip)
                current_time += a_clip.duration
                
                # Brief pause between cards
                if i < len(flashcards) - 1:
                    transition = self.create_transition_clip()
                    clips.append(transition)
                    current_time += transition.duration
            
            print("Combining all clips...")
            # Combine all clips
            final_video = concatenate_videoclips(clips)
            
            print(f"Writing final video to {output_file}...")
            # Write video with reduced verbosity
            final_video.write_videofile(
                output_file,
                fps=24,
                codec='libx264',
                audio_codec='aac',
                temp_audiofile=os.path.join(self.temp_dir, 'temp-audio.m4a'),
                remove_temp=True,
                preset='medium',
                verbose=False,  # Suppress verbose output
                logger=None  # Disable progress bars
            )
            
            return timestamps
    
    def create_question_clip(self, question, number):
        """Create a question slide with audio - dynamic duration"""
        
        # First create the audio to get its duration
        temp_filename = os.path.join(self.temp_dir, f"q_{number}.mp3")
        # New:
        spoken_text = self.preprocess_text_for_tts(f"Question {number}. {question}")
        tts = gTTS(spoken_text, lang='en', slow=False)
        
        # tts = gTTS(f"Question {number}. {question}", lang='en', slow=False)
        tts.save(temp_filename)
        audio_clip = AudioFileClip(temp_filename)
        
        # Use actual audio duration plus a small buffer
        duration = audio_clip.duration + 1.0  # Add 1 second buffer
        
        # Create visual
        img = Image.new('RGB', (self.width, self.height), color='#1e3d59')
        draw = ImageDraw.Draw(img)
        
        draw.text((self.width//2, 200), f"Question {number}", 
                 fill='#f5f0e1', font=self.font_question_small, anchor="mm")
        
        self.draw_wrapped_text(draw, question, self.font_question_large, '#ffffff')
        
        # Convert to video clip with correct duration
        img_array = np.array(img)
        video_clip = ImageClip(img_array).set_duration(duration)
        
        # Combine
        video_with_audio = video_clip.set_audio(audio_clip)
        
        return video_with_audio
    
    def create_answer_clip(self, answer, number):
        """Create answer slide with audio - dynamic duration"""
        
        # First create the audio to get its duration
        temp_filename = os.path.join(self.temp_dir, f"a_{number}.mp3")

        # New:
        spoken_text = self.preprocess_text_for_tts(f"The answer is: {answer}")
        tts = gTTS(spoken_text, lang='en', slow=True)
        
        # tts = gTTS(f"The answer is: {answer}", lang='en', slow=True)
        tts.save(temp_filename)
        audio_clip = AudioFileClip(temp_filename)
        
        # Use actual audio duration plus a buffer
        duration = audio_clip.duration + 1.5  # Add 1.5 second buffer for answers
        
        # Create visual
        img = Image.new('RGB', (self.width, self.height), color='#27ae60')
        draw = ImageDraw.Draw(img)
        
        draw.text((self.width//2, 200), "ANSWER", 
                 fill='#ffffff', font=self.font_answer_small, anchor="mm")
        
        self.draw_wrapped_text(draw, answer, self.font_answer_large, '#ffffff')
        
        # Convert to video with correct duration
        img_array = np.array(img)
        video_clip = ImageClip(img_array).set_duration(duration)
        
        # Combine
        video_with_audio = video_clip.set_audio(audio_clip)
        
        return video_with_audio
    
    def create_pause_clip(self, duration=3):
        """Create a thinking pause with countdown"""
        
        clips = []
        for i in range(duration, 0, -1):
            img = Image.new('RGB', (self.width, self.height), color='#2c3e50')
            draw = ImageDraw.Draw(img)
            
            draw.text((self.width//2, self.height//2), str(i), 
                     fill='#ecf0f1', font=self.font_countdown, anchor="mm")
            draw.text((self.width//2, self.height//2 + 150), "Think about it...", 
                     fill='#95a5a6', font=self.font_countdown_text, anchor="mm")
            
            img_array = np.array(img)
            clip = ImageClip(img_array).set_duration(1)
            clips.append(clip)
        
        return concatenate_videoclips(clips)
    
    def create_transition_clip(self, duration=0.5):
        """Brief transition between cards"""
        img = Image.new('RGB', (self.width, self.height), color='#34495e')
        img_array = np.array(img)
        return ImageClip(img_array).set_duration(duration)
    
    def create_text_clip(self, text, duration=3):
        """Create a simple text clip"""
        
        # Create audio first
        temp_filename = os.path.join(self.temp_dir, "intro.mp3")
        tts = gTTS(text, lang='en')
        tts.save(temp_filename)
        audio = AudioFileClip(temp_filename)
        
        # Adjust duration if audio is longer
        duration = max(duration, audio.duration + 0.5)
        
        img = Image.new('RGB', (self.width, self.height), color='#2c3e50')
        draw = ImageDraw.Draw(img)
        
        draw.text((self.width//2, self.height//2), text, 
                 fill='#ffffff', font=self.font_intro, anchor="mm")
        
        img_array = np.array(img)
        video = ImageClip(img_array).set_duration(duration)
        video_with_audio = video.set_audio(audio)
        
        return video_with_audio
    
    def draw_wrapped_text(self, draw, text, font, color):
        """Draw text with word wrapping"""
        words = text.split()
        lines = []
        current_line = []
        
        max_width = self.width - 200
        
        for word in words:
            current_line.append(word)
            test_line = ' '.join(current_line)
            
            bbox = draw.textbbox((0, 0), test_line, font=font)
            text_width = bbox[2] - bbox[0]
            
            if text_width > max_width:
                if len(current_line) > 1:
                    current_line.pop()
                    lines.append(' '.join(current_line))
                    current_line = [word]
                else:
                    lines.append(test_line)
                    current_line = []
        
        if current_line:
            lines.append(' '.join(current_line))
        
        bbox = draw.textbbox((0, 0), "Test", font=font)
        line_height = bbox[3] - bbox[1] + 20
        
        total_height = len(lines) * line_height
        y_position = (self.height - total_height) // 2
        
        for line in lines:
            draw.text((self.width//2, y_position), line, 
                     fill=color, font=font, anchor="mt")
            y_position += line_height
    
    def format_time(self, seconds):
        """Format seconds to YouTube timestamp format"""
        mins = int(seconds) // 60
        secs = int(seconds) % 60
        return f"{mins:02d}:{secs:02d}"


def load_flashcards_from_csv(csv_file_path):
    """Load flashcards from a CSV file"""
    flashcards = []
    
    try:
        with open(csv_file_path, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            
            if 'Question' not in reader.fieldnames or 'Answer' not in reader.fieldnames:
                print(f"Error: CSV must have 'Question' and 'Answer' columns")
                print(f"Found columns: {reader.fieldnames}")
                return None
            
            for row_num, row in enumerate(reader, start=1):
                question = row['Question'].strip()
                answer = row['Answer'].strip()
                
                if question and answer:
                    flashcards.append({
                        'question': question,
                        'answer': answer
                    })
        
        print(f"✓ Loaded {len(flashcards)} flashcards from {csv_file_path}")
        return flashcards
        
    except FileNotFoundError:
        print(f"Error: CSV file not found: {csv_file_path}")
        return None
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None


def create_video_from_csv(csv_file_path):
    """Create a video from a CSV file of flashcards"""
    flashcards = load_flashcards_from_csv(csv_file_path)
    
    if not flashcards:
        print("No flashcards loaded. Exiting.")
        return
    
    csv_filename = os.path.basename(csv_file_path)
    output_filename = os.path.splitext(csv_filename)[0] + ".mp4"
    
    print(f"Creating video: {output_filename}")
    
    creator = FlashcardVideoCreator()
    timestamps = creator.create_study_video(flashcards, output_filename)
    
    print("\nPaste these timestamps in your YouTube description:")
    print("=" * 50)
    for timestamp in timestamps:
        print(timestamp)
    
    print(f"\n✓ Video created successfully: {output_filename}")
    print(f"  Total flashcards: {len(flashcards)}")


if __name__ == "__main__":
    csv_file = "hypothesis_testing.csv"
    create_video_from_csv(csv_file)

✓ Loaded 50 flashcards from hypothesis_testing.csv
Creating video: hypothesis_testing.mp4
✓ Using Windows font: cambriab.ttf
Creating intro...
Processing card 1/50...
Processing card 2/50...
Processing card 3/50...
Processing card 4/50...
Processing card 5/50...
Processing card 6/50...
Processing card 7/50...
Processing card 8/50...
Processing card 9/50...
Processing card 10/50...
Processing card 11/50...
Processing card 12/50...
Processing card 13/50...
Processing card 14/50...
Processing card 15/50...
Processing card 16/50...
Processing card 17/50...
Processing card 18/50...
Processing card 19/50...
Processing card 20/50...
Processing card 21/50...
Processing card 22/50...
Processing card 23/50...
Processing card 24/50...
Processing card 25/50...
Processing card 26/50...
Processing card 27/50...
Processing card 28/50...
Processing card 29/50...
Processing card 30/50...
Processing card 31/50...
Processing card 32/50...
Processing card 33/50...
Processing card 34/50...
Processing card 

# The next cell creates a wrapper for the code above which creates the video 
# and programmatically uploads it to YouTube using the YouTube API.
# You can now quickly create an audio plus video flashcard set to use on the go!

In [12]:

# Path to Authenticate on YouTube
def get_credentials_path():
    home = Path.home()
    creds_dir = home / '.credentials'
    creds_dir.mkdir(exist_ok=True)
    return str(creds_dir / 'client_secret.json') #Path to your secret YouTube credentials



class YouTubeUploader:
    def __init__(self, client_secrets_file=None):  # Changed parameter
        # If no path provided, use the default secure location
        if client_secrets_file is None:
            self.client_secrets_file = get_credentials_path()  # Call the function
        else:
            self.client_secrets_file = client_secrets_file
        self.credentials = None
        self.youtube = None
        self.authenticate()
    
    def authenticate(self):
        """Authenticate and create YouTube service object"""
        SCOPES = ['https://www.googleapis.com/auth/youtube.upload']
        
        # Token file stores the user's access and refresh tokens
        token_file = 'token.pickle'
        
        # Check if we have saved credentials
        if os.path.exists(token_file):
            with open(token_file, 'rb') as token:
                self.credentials = pickle.load(token)
        
        # If there are no (valid) credentials available, let the user log in
        if not self.credentials or not self.credentials.valid:
            if self.credentials and self.credentials.expired and self.credentials.refresh_token:
                self.credentials.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    self.client_secrets_file, SCOPES)
                self.credentials = flow.run_local_server(port=0)
            
            # Save the credentials for the next run
            with open(token_file, 'wb') as token:
                pickle.dump(self.credentials, token)
        
        self.youtube = build('youtube', 'v3', credentials=self.credentials)
        print("✓ YouTube authentication successful")
    
    def upload_video(self, 
                     video_file, 
                     title, 
                     description='', 
                     tags=None, 
                     category_id='27',  # Education category
                     privacy_status='unlisted',
                     thumbnail=None):
        """
        Upload video to YouTube
        
        Args:
            video_file: Path to video file
            title: Video title (max 100 chars)
            description: Video description (max 5000 chars)
            tags: List of tags
            category_id: YouTube category (27 = Education)
            privacy_status: 'private', 'unlisted', or 'public'
            thumbnail: Path to thumbnail image (optional)
        """
        
        if tags is None:
            tags = []
        
        # Ensure title isn't too long
        if len(title) > 100:
            title = title[:97] + "..."
        
        body = {
            'snippet': {
                'title': title,
                'description': description,
                'tags': tags,
                'categoryId': category_id
            },
            'status': {
                'privacyStatus': privacy_status,
                'selfDeclaredMadeForKids': False
            }
        }
        
        # Call the API's videos.insert method to create and upload the video
        insert_request = self.youtube.videos().insert(
            part=','.join(body.keys()),
            body=body,
            media_body=MediaFileUpload(
                video_file, 
                chunksize=-1, 
                resumable=True
            )
        )
        
        return self.resumable_upload(insert_request, video_file)
    
    def resumable_upload(self, insert_request, video_file):
        """Handle resumable upload with retry logic"""
        
        response = None
        error = None
        retry = 0
        max_retries = 3
        
        print(f"Uploading {video_file}...")
        
        while response is None:
            try:
                status, response = insert_request.next_chunk()
                if response is not None:
                    if 'id' in response:
                        video_id = response['id']
                        video_url = f"https://youtube.com/watch?v={video_id}"
                        print(f"✓ Upload successful!")
                        print(f"  Video ID: {video_id}")
                        print(f"  URL: {video_url}")
                        return {
                            'id': video_id,
                            'url': video_url,
                            'response': response
                        }
                    else:
                        print(f"Upload failed: {response}")
                        return None
                        
                if status:
                    print(f"  Upload progress: {int(status.progress() * 100)}%")
                    
            except HttpError as e:
                if e.resp.status in [500, 502, 503, 504]:
                    error = f"HTTP {e.resp.status} error: {e.content}"
                    retry += 1
                    if retry > max_retries:
                        print(f"Max retries exceeded. Upload failed: {error}")
                        return None
                    
                    wait_time = 2 ** retry + random.random()
                    print(f"Error occurred. Waiting {wait_time:.1f} seconds before retry...")
                    time.sleep(wait_time)
                else:
                    raise e
            
            except Exception as e:
                print(f"An error occurred: {e}")
                return None
        
        return None

def create_and_upload_video(csv_file_path, upload_to_youtube=True, recreate_video=False):
    """Enhanced function that creates video and optionally uploads to YouTube
    
    Args:
        csv_file_path: Path to CSV file with flashcards
        upload_to_youtube: Whether to upload to YouTube after creation
        recreate_video: Force recreation even if video exists
    """
    
    csv_filename = os.path.basename(csv_file_path)
    output_filename = os.path.splitext(csv_filename)[0] + ".mp4"
    video_title = os.path.splitext(csv_filename)[0].replace('_', ' ').title()
    
    # Check if video already exists
    video_exists = os.path.exists(output_filename)
    
    if video_exists and not recreate_video:
        print(f"✓ Video already exists: {output_filename}")
        file_size_mb = os.path.getsize(output_filename) / (1024 * 1024)
        print(f"  File size: {file_size_mb:.2f} MB")
        
        # Ask user what to do
        user_choice = input("\nOptions:\n1. Upload existing video\n2. Recreate video\n3. Skip\nChoice (1/2/3): ").strip()
        
        if user_choice == '2':
            video_exists = False  # Will recreate below
        elif user_choice == '3':
            print("Skipping...")
            return output_filename
        # Choice 1 continues to upload section
    
    # Create video if it doesn't exist or user chose to recreate
    if not video_exists or recreate_video:
        # Load flashcards
        flashcards = load_flashcards_from_csv(csv_file_path)
        
        if not flashcards:
            print("No flashcards loaded. Exiting.")
            return None
        
        print(f"Creating video: {output_filename}")
        
        creator = FlashcardVideoCreator()
        timestamps = creator.create_study_video(flashcards, output_filename)
        
        print(f"\n✓ Video created successfully: {output_filename}")
    else:
        # Video exists and we're just uploading - need to generate timestamps for description
        print("Generating timestamps for existing video...")
        flashcards = load_flashcards_from_csv(csv_file_path)
        if not flashcards:
            print("Warning: Could not load flashcards for description")
            timestamps = []
        else:
            # Approximate timestamps (won't be exact without recreating)
            timestamps = []
            current_time = 3  # Intro
            for i, card in enumerate(flashcards):
                timestamps.append(f"{format_time(current_time)} Question {i+1}: {card['question'][:50]}")
                current_time += 10  # Rough estimate per card
    
    # Format description with timestamps
    description = f"Study flashcards for {video_title}\n\n"
    if timestamps:
        description += "Timestamps:\n"
        for timestamp in timestamps:
            description += f"{timestamp}\n"
    description += "\n\nCreated with automated flashcard video generator."
    
    # Upload to YouTube if requested
    if upload_to_youtube:
        print("\n" + "="*50)
        print("Uploading to YouTube...")
        
        try:
            uploader = YouTubeUploader()  # Fixed to use default path
            
            result = uploader.upload_video(
                video_file=output_filename,
                title=f"{video_title} - Study Flashcards",
                description=description,
                tags=['education', 'flashcards', 'study', 'learning', video_title.lower()],
                category_id='27',  # Education
                privacy_status='unlisted'
            )
            
            if result:
                print(f"\n🎉 Video available at: {result['url']}")
                
                # Save URL to file for reference
                with open('uploaded_videos.txt', 'a') as f:
                    f.write(f"{output_filename}: {result['url']}\n")
            
        except Exception as e:
            print(f"Upload failed: {e}")
            print("Video saved locally but not uploaded.")
    
    return output_filename

def format_time(seconds):
    """Helper function for timestamp formatting"""
    mins = int(seconds) // 60
    secs = int(seconds) % 60
    return f"{mins:02d}:{secs:02d}"

# Usage
if __name__ == "__main__":
    # Single video with upload
    csv_file = "hypothesis_testing.csv"
    create_and_upload_video(csv_file, upload_to_youtube=True)
    
    # Or batch process multiple files
    # csv_files = ["spanish_vocabulary.csv", "biology_terms.csv"]
    # for csv_file in csv_files:
    #     create_and_upload_video(csv_file, upload_to_youtube=True)
    #     time.sleep(60)  # Wait between uploads to avoid rate limits

✓ Video already exists: hypothesis_testing.mp4
  File size: 15.98 MB



Options:
1. Upload existing video
2. Recreate video
3. Skip
Choice (1/2/3):  1


Generating timestamps for existing video...
✓ Loaded 50 flashcards from hypothesis_testing.csv

Uploading to YouTube...
✓ YouTube authentication successful
Uploading hypothesis_testing.mp4...
✓ Upload successful!
  Video ID: 4gioGZFbnPw
  URL: https://youtube.com/watch?v=4gioGZFbnPw

🎉 Video available at: https://youtube.com/watch?v=4gioGZFbnPw
